In [1]:
from typing import *

In [2]:
import itertools

LEGAL_TERMS = {
    "company": ("Co", ),
    "limited partnership": ("LP", ),
    "limited liability partnership": ("LLP", ),
    "limited liability limited partnership": ("LLLP",),
    "limited liability companies": ("LLC", "LC", "Ltd. Co.", "Ltd."),
    "limited liability company": ("LLC", "LC", "Ltd. Co.", "Ltd."),
    "professional limited liability company": ("PPLC", ),
    "association": (),
    "organization": (),
}
ALREADY_ABBREVIATIONS = set(itertools.chain(*LEGAL_TERMS.values()))

WEAK_WORDS = {"of", "the", "for"}

In [3]:
def abbreviations_as_first_letters(words: List[str]):
    if not words:
        yield ""
        return

    for abbr in abbreviations_as_first_letters(words[1:]):
        word = words[0]
        if word in ALREADY_ABBREVIATIONS:
            yield abbr
            continue
        if word in WEAK_WORDS:
            yield abbr
        if word == "and":
            yield abbr
            yield "&" + abbr
        yield word[0].upper() + abbr


def split_legal_terms(name: str) -> Generator[str, None, None]:
    for legal_term in LEGAL_TERMS:
        if name.endswith(legal_term):
            yield name[:-len(legal_term)], legal_term

            
def abbreviations_with_legal_terms(name: str) -> Generator[str, None, None]:
    for main_name, suffix in split_legal_terms(name):
        for abbreviation in abbreviations_as_first_letters(main_name.split()):
            yield abbreviation
            yield abbreviation + " " + suffix
            for possible_suffix in LEGAL_TERMS[suffix]:
                yield abbreviation + " " + possible_suffix
    yield from abbreviations_as_first_letters(name.split())
    
    
def generate(name: str) -> Generator[str, None, None]:
    for abbreviation in abbreviations_with_legal_terms(name.lower()):
        if len(abbreviation) < 2:
            continue

        yield abbreviation.lower()
        if len(set(abbreviation)) == 1:
            yield str(len(abbreviation)) + abbreviation[0].lower()
        
    

In [4]:
test_cases = {
    "Government Employees Insurance Company": "GEICO",  #maybe special case?
    "Head, Heart, Hands, Health": "4H",
    "American Consultants League": "ACL",
    "Amyotrophic Lateral Sclerosis Association": "ALS Association",
    "Council of Actions United for Service Efforts": "CAUSE",
    "Conference of Minority Public Administrators": "COMPA",
    "The Minnesota Mining and Manufacturing Company": "3M",
}

special_cases = {
    "International Crime Police Organization": "INTERPOL",
    "United Nations Children’s Fund": "UNICEF",
    "Henry and Richard Block": "H&R Block",
    "Shoulder of Pork and Ham": "SPAM",
    "Transport for Elderly and Disabled Persons": "TRANSED",
}

In [5]:
ALL = True

for test in test_cases:
    possibilities = set(generate(test))
    should_be = test_cases[test].lower()
    if ALL or not should_be in possibilities:
        print(test.ljust(60), ",".join(possibilities))

Government Employees Insurance Company                       gei company,gei co,gei,geic
Head, Heart, Hands, Health                                   hhhh,4h
American Consultants League                                  acl
Amyotrophic Lateral Sclerosis Association                    alsa,als association,als
Council of Actions United for Service Efforts                caufse,coause,coaufse,cause
Conference of Minority Public Administrators                 compa,cmpa
The Minnesota Mining and Manufacturing Company               tmmam co,mmm co,tmm&m company,tmm&m co,tmmmc,mm&mc,mmm,mm&m company,mmamc,3m,mmmc,tmmm company,mmm company,tmmam company,mm&m co,tmmam,tmm&mc,mmam,tmmm co,tmmamc,tmm&m,tmmm,mmam company,mm&m,mmam co


In [6]:
for test in special_cases:
    possibilities = set(generate(test))
    should_be = special_cases[test].lower()
    print(test.ljust(60), should_be.ljust(20), ",".join(possibilities))

International Crime Police Organization                      interpol             icpo,icp,icp organization
United Nations Children’s Fund                               unicef               uncf
Henry and Richard Block                                      h&r block            h&rb,hrb,harb
Shoulder of Pork and Ham                                     spam                 soph,sp&h,sph,sop&h,sopah,spah
Transport for Elderly and Disabled Persons                   transed              tedp,tfe&dp,tfedp,te&dp,teadp,tfeadp


In [7]:
print(list(generate("GOOGLE")))   #no sensible abbreviation

[]
